In [ ]:
import wget
import pandas as pd
import dash

In [ ]:
# import the required libraries
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
data = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [ ]:
spacex_df = pd.read_csv(data)
spacex_df.columns

In [ ]:
px.pie(spacex_df, values='class', names='Launch Site')

In [ ]:
launch_sites = [i for i in set(spacex_df['Launch Site'])]
launch_sites

In [ ]:
spacex_success_df = spacex_df[spacex_df['class'] == 1].reset_index()
spacex_success_df.head()

In [ ]:
# Assign minimum and maximum payloads
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [ ]:
payload_range = range(7000, 10000)

In [ ]:
fdf = spacex_df[spacex_df['Payload Mass (kg)'].isin(payload_range)].reset_index()

payload_range =  range(5000, 10000)
fdf = spacex_df[spacex_df['Payload Mass (kg)'].apply(lambda x:x in payload_range).reset_index()]

In [ ]:
fig = px.scatter(x='Payload Mass (kg)', y='class', data_frame=spacex_df, color = 'Booster Version Category',
                        title = 'Correlation between payload and success for site')
fig

In [ ]:
max_payload, min_payload

In [ ]:
# Create dash app
app = dash.Dash(__name__)

In [ ]:
# Create the app layout
app.layout = html.Div(children = [html.H1('SpaceX Launch Records Dashboard',
                                         style = {'textAlign': 'center', 
                                                 'color': '#503D36',
                                                 'font-size': '40'}),
                                  # TASK 1: Add a dropdown list to enable Launch Site selection
                                  # The default select value is for ALL sites
                                  # dcc.Dropdown(id='site-dropdown',...)
                                  dcc.Dropdown(id='site-dropdown', 
                                              options = [
                                                  {'label': 'All Sites', 'value': 'ALL'},
                                                  {'label': launch_sites[0] , 'value': launch_sites[0]},
                                                  {'label': launch_sites[1] , 'value': launch_sites[1]},
                                                  {'label': launch_sites[2] , 'value': launch_sites[2]},
                                                  {'label': launch_sites[3] , 'value': launch_sites[3]}
                                              ],
                                               placeholder = 'Select Launch Site',
                                               searchable = True,
                                              style = {'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}),
                                 html.Br(),
                                  
                                  # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                 html.Div(dcc.Graph(id='success-pie-chart')),
                                 html.Br(),
                                 
                                  html.P('Payload range (kg)'),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                  dcc.RangeSlider(min=min_payload, max=max_payload, id = 'payload-slider'),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                  html.Div(dcc.Graph(id='success-payload-scatter-chart')),                                 
                                 ])


In [ ]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

# computations to return the pie chart
def get_pie_chart(entered_site):
    # condition on whether value is 'ALL' or specific launch site
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values = 'class', names = 'Launch Site', title = 'Total Success Launches by Site')
    
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site].reset_index()
        fig = px.pie(filtered_df, names = 'class', title = 'Success Launches vrs Failed Launches on Site {}'.format(entered_site))
    
    return fig


In [ ]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')])

# Computations to callback fundtions to return the scatter graphs
def get_scatter_plot(launch_site, payload_range):
    
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].isin(payload_range)].reset_index()
    
    if launch_site == 'ALL':
        if payload_range:
            fig = px.scatter(x='Payload Mass (kg)', y='class', data_frame=spacex_df, color = 'Booster Version Category',
                        title='Correlation between payload and success for all sites')
        else:
            fig = px.scatter(x='Payload Mass (kg)', y='class', data_frame=filtered_df, color='Booster Version Category',
                            title='Correlation between payload and success for all sites')
    
    else:
        filtered_df2 = spacex_df[spacex_df['Launch Site'] == launch_site].reset_index()
        if payload_range:
            fig = px.scatter(x='Payload Mass (kg)', y='class', data_frame=filtered_df2, color = 'Booster Version Category',
                        title = 'Correlation between payload and success for {} site'.format(launch_site))
        else:
            filtered_df3 = filtered_df[spacex_df['Launch Site'] == launch_site].reset_index()
            fig = px.scatter(x='Payload Mass (kg)', y='class', data_frame=filtered_df3, color = 'Booster Version Category',
                        title = 'Correlation between payload and success for {} site'.format(launch_site))
    
    return fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

In [ ]:
# px.scatter(x='Payload Mass (kg)', y = 'class', data_frame=spacex_df, color='Booster Version Category')